<a href="https://colab.research.google.com/github/JPOrellana/LAB5_VEC/blob/main/Laboratorio_4_Vectorizacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NOMBRES:

APELLIDOS:

CARNE:

FECHA:

**Ejercicio 1**
Cree un corpus a su gusto como el visto en clase, cálcule PPMI, pero aplicando Lapace Smoothing.

**Ejercicio 2**

Crear 10 analigias validas basadas en su texto, tome en cuenta que es necesario utilizar el codigo visto en clase y ajuste su corpus para poder lograr obtener todas las analogias